In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if ".pkl" in filename:
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S14/S14.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S11/S11.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S13/S13.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S10/S10.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S8/S8.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S5/S5.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S7/S7.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S9/S9.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S15/S15.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S2/S2.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S6/S6.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S3/S3.pkl
/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD

In [2]:
# /kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S14/S14.pkl
import pandas as pd

# Pfad zur Datei
file_path = '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S14/S14.pkl'

# Einlesen der Pickle-Datei
with open(file_path, 'rb') as file:
    data = pd.read_pickle(file)

# Anzeigen der Struktur (z. B. Keys, falls es ein Dictionary ist)
print(type(data))
print(data.keys())


<class 'dict'>
dict_keys(['signal', 'label', 'subject'])


In [3]:
import numpy as np
import collections

# Sensorsignale und Labels extrahieren
signals = data['signal']  # dict mit 'eda', 'ecg', etc.
labels = data['label']    # NumPy-Array (z. B. [1 1 1 2 2 2 ...])

print(signals.keys())
# Beispiel: EDA-Signal


# Filter: nur Zeilen mit label == 2
#type(data[labels == 2])
#cells_with_stress = data[labels == 2]
#type(cells_with_stress)
# Sortieren: z. B. nach EDA-Werten (aufsteigend)
#cells_with_stress_sorted = np.sort(cells_with_stress, axis=0)

#print("EDA-Signale mit Label=2, sortiert:\n", cells_with_stress_sorted)

import collections

# Häufigkeiten zählen
label_counts = dict(collections.Counter(labels))

# Label-Bedeutungen laut WESAD (inkl. erweiterter Interpretation)
label_meanings = {
    0: "Keine Annotation / Übergang",
    1: "Baseline (Ruhe)",
    2: "Stressphase (z. B. Rechenaufgabe)",
    3: "Amusement (z. B. lustiges Video)",
    4: "Entspannung / Meditation (optional)",
    5: "Fragebogen / Pause (optional)",
    6: "Vorbereitung / Setup (optional)",
    7: "Technischer Übergang / unbekannt (optional)"
}

# Übersicht ausgeben
for label, count in sorted(label_counts.items()):
    meaning = label_meanings.get(label, "Unbekannt")
    print(f"Label {label}: {count} Einträge – {meaning}")

print(data['subject'])

dict_keys(['chest', 'wrist'])
Label 0: 1648499 Einträge – Keine Annotation / Übergang
Label 1: 826000 Einträge – Baseline (Ruhe)
Label 2: 472500 Einträge – Stressphase (z. B. Rechenaufgabe)
Label 3: 260401 Einträge – Amusement (z. B. lustiges Video)
Label 4: 555800 Einträge – Entspannung / Meditation (optional)
Label 5: 44100 Einträge – Fragebogen / Pause (optional)
Label 6: 38500 Einträge – Vorbereitung / Setup (optional)
Label 7: 37800 Einträge – Technischer Übergang / unbekannt (optional)
S14


In [4]:
data['signal']['wrist']

{'ACC': array([[ -23.,   31.,  127.],
        [-128.,  -31.,   62.],
        [ -54.,  -58.,   26.],
        ...,
        [ -47.,  -45.,    6.],
        [ -47.,  -44.,    8.],
        [ -47.,  -45.,    8.]]),
 'BVP': array([[  -8.27],
        [  -9.52],
        [ -10.68],
        ...,
        [-123.97],
        [-160.08],
        [-195.94]]),
 'EDA': array([[0.277864],
        [0.266351],
        [0.195992],
        ...,
        [0.080859],
        [0.060391],
        [0.175257]]),
 'TEMP': array([[31.97],
        [31.97],
        [31.97],
        ...,
        [31.21],
        [31.21],
        [31.21]])}

## Daten sortieren wo nur Stress = 2 ist

Es sind viel zu wenige Daten für Stress. Oben sind es nich 470 Tsd unten nur noch 2700?!

In [5]:
from scipy.signal import resample


labels = data['label']  # Länge z. B. 3883600
eda = data['signal']['wrist']['EDA']  # Länge z. B. 22192

print(f"Label length: {len(labels)} | EDA length: {len(eda)}")

# Labels auf die Länge der EDA-Daten bringen
labels_resampled = resample(labels.astype(float), len(eda))
labels_resampled = np.round(labels_resampled).astype(int)

# Maske anwenden
mask = labels_resampled == 2
eda_label2 = eda[mask]

# In DataFrame
df_eda_label2 = pd.DataFrame({
    'EDA': eda_label2[:, 0],
    'Label': [2] * len(eda_label2)
})

print(df_eda_label2)


Label length: 3883600 | EDA length: 22192
           EDA  Label
0     0.339268      2
1     0.337989      2
2     0.336710      2
3     0.337989      2
4     0.337989      2
...        ...    ...
2697  0.336710      2
2698  0.335430      2
2699  0.316242      2
2700  0.314962      2
2701  0.286819      2

[2702 rows x 2 columns]


### Baseline

In [6]:
from scipy.signal import resample


labels = data['label']  # Länge z. B. 3883600
eda = data['signal']['wrist']['EDA']  # Länge z. B. 22192

print(f"Label length: {len(labels)} | EDA length: {len(eda)}")

# Labels auf die Länge der EDA-Daten bringen
labels_resampled = resample(labels.astype(float), len(eda))
labels_resampled = np.round(labels_resampled).astype(int)

# Maske anwenden
mask = labels_resampled == 1
eda_label2 = eda[mask]

# In DataFrame
df_eda_label2 = pd.DataFrame({
    'EDA': eda_label2[:, 0],
    'Label': [1] * len(eda_label2)
})

print(df_eda_label2)


Label length: 3883600 | EDA length: 22192
           EDA  Label
0     0.303449      1
1     0.295773      1
2     0.291936      1
3     0.293215      1
4     0.285539      1
...        ...    ...
4721  0.339268      1
4722  0.335430      1
4723  0.321359      1
4724  0.313683      1
4725  0.321359      1

[4726 rows x 2 columns]


### Amusement

In [7]:
from scipy.signal import resample


labels = data['label']  # Länge z. B. 3883600
eda = data['signal']['wrist']['EDA']  # Länge z. B. 22192

print(f"Label length: {len(labels)} | EDA length: {len(eda)}")

# Labels auf die Länge der EDA-Daten bringen
labels_resampled = resample(labels.astype(float), len(eda))
labels_resampled = np.round(labels_resampled).astype(int)

# Maske anwenden
mask = labels_resampled == 3
eda_label2 = eda[mask]

# In DataFrame
df_eda_label2 = pd.DataFrame({
    'EDA': eda_label2[:, 0],
    'Label': [3] * len(eda_label2)
})

print(df_eda_label2)


Label length: 3883600 | EDA length: 22192
           EDA  Label
0     0.303449      3
1     0.314962      3
2     0.316242      3
3     0.317521      3
4     0.316242      3
...        ...    ...
1484  0.313683      3
1485  0.313683      3
1486  0.312404      3
1487  0.307287      3
1488  0.306008      3

[1489 rows x 2 columns]


# Random Forest

In [8]:
import pandas as pd
import numpy as np
from scipy.signal import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Pfad zur Datei
file_path = ['/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S14/S14.pkl',
             '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S13/S13.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S10/S10.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S5/S5.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S7/S7.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S9/S9.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S15/S15.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S2/S2.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S6/S6.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S3/S3.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S4/S4.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S16/S16.pkl',
            '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S17/S17.pkl'
            ]

features = []
window_labels = []

for file in file_path:
# Laden
    with open(file, 'rb') as f:
        data = pd.read_pickle(f)
    
    # Zugriff auf Wrist-Daten
    wrist = data['signal']['wrist']
    eda = wrist['EDA']      # shape (n_samples, 1)
    temp = wrist['TEMP']    # shape (n_samples, 1)
    acc = wrist['ACC']      # shape (n_samples, 3)
    
    # Label (ursprünglich 700 Hz, z. B. 3.8 Mio lang)
    labels = data['label']
    
    # Resampling der Labels auf Länge von EDA (64 Hz)
    labels_resampled = resample(labels.astype(float), len(eda))
    labels_resampled = np.round(labels_resampled).astype(int)
    
    # Features kombinieren (z. B. EDA, TEMP, ACC)
    from scipy.signal import resample
    
    # Resample ACC auf Länge von EDA (z. B. 22192 Zeilen)
    acc_resampled = resample(acc, len(eda))
    
    X_raw = np.hstack([eda, temp, acc_resampled])  # shape: (n_samples, 6)

    X_raw
    
    # Sliding Window-Feature-Bildung (z. B. 60s bei 64 Hz = 384 Samples)
    window_size = 384  # 60s Fenster
    step_size = 384    # ohne Überschneidung

    
    for start in range(0, len(X_raw) - window_size, step_size):
        end = start + window_size
        window = X_raw[start:end]
        label_window = labels_resampled[start:end]
        
        # Nur Fenster mit sinnvollen Labels behalten (z. B. 1 = Ruhe, 2 = Stress)
        if np.all(np.isin(label_window, [1, 2])):
            # Feature-Vektor: Mittelwert + Std jeder Sensor-Spalte
            feature_vector = np.concatenate([window.mean(axis=0), window.std(axis=0)])
            features.append(feature_vector)
            
            # Mehrheits-Label als Klassenlabel
            majority_label = np.bincount(label_window).argmax()
            window_labels.append(majority_label)

# In Arrays umwandeln
X = np.array(features)
y = np.array(window_labels)

print(f"Datensätze: {X.shape}, Klassenverteilung: {np.bincount(y)}")




Datensätze: (227, 10), Klassenverteilung: [  0 149  78]


In [9]:
X_raw[8]

array([  1.968359  ,  33.34      , -49.9914787 , -38.16717886,
         5.37479379])

In [10]:
# Modell trainieren auf 80:20 Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,     # 20% Testdaten
    random_state=42,   # Reproduzierbarkeit
)
clf_testdaten = RandomForestClassifier(n_estimators=105, random_state=42)
clf_testdaten.fit(X_train, y_train)


# Test auf 80:20 Split
y_pred_test = clf_testdaten.predict(X_test)
print(classification_report(y_test, y_pred_test, digits=3))

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def finde_bestes_n_estimators(X_train, X_test, y_train, y_test, 
                              n_min=10, n_max=200, step=10, average="weighted"):
    ergebnisse = {}
    
    for n in range(n_min, n_max+1, step):
        clf = RandomForestClassifier(n_estimators=n, random_state=42)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        acc = accuracy_score(y_test, y_pred)
        f1  = f1_score(y_test, y_pred, average=average)
        
        ergebnisse[n] = {"accuracy": acc, "f1_score": f1}
        #print(f"n_estimators={n}: Accuracy={acc:.3f}, F1={f1:.3f}")
    
    # Auswahl nach bestem F1-Score
    best_n = max(ergebnisse, key=lambda k: ergebnisse[k]["f1_score"])
    best_vals = ergebnisse[best_n]
    
    print(f"\n➡️ Bestes n_estimators: {best_n} "
          f"mit Accuracy={best_vals['accuracy']:.3f}, F1={best_vals['f1_score']:.3f}")
    
    return best_n, ergebnisse


finde_bestes_n_estimators(X_train, X_test, y_train, y_test, n_min=10, n_max=200, step=1)


import joblib  # für Speichern/Laden

# Modell trainieren auf kombinierten Trainingsdaten
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)
# ✅ Speichern des gesamten Modells ohne Testdatensplit
joblib.dump(clf, "/kaggle/working/random_forest_model.pkl")


              precision    recall  f1-score   support

           1      0.853     1.000     0.921        29
           2      1.000     0.706     0.828        17

    accuracy                          0.891        46
   macro avg      0.926     0.853     0.874        46
weighted avg      0.907     0.891     0.886        46


➡️ Bestes n_estimators: 15 mit Accuracy=0.913, F1=0.910


['/kaggle/working/random_forest_model.pkl']

## Daten der S8 ziehen und einsetzen!

In [11]:
import pandas as pd
import numpy as np
from scipy.signal import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 📁 Pfad zur Datei (S12)
file_path_S12 = '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S8/S8.pkl'

# 🔄 Daten laden
with open(file_path_S12, 'rb') as f:
    data_S12 = pd.read_pickle(f)

# 📦 Zugriff auf Wrist-Signale
wrist_S12 = data_S12['signal']['wrist']
eda_S12 = wrist_S12['EDA']
temp_S12 = wrist_S12['TEMP']
acc_S12 = wrist_S12['ACC']

# 🏷️ Labels (ursprünglich 700 Hz)
labels_S12 = data_S12['label']

# 🔁 Resampling der Labels auf EDA-Länge (64 Hz)
labels_resampled_S12 = resample(labels_S12.astype(float), len(eda_S12))
labels_resampled_S12 = np.round(labels_resampled_S12).astype(int)

# 🔁 Resample ACC auf EDA-Länge
acc_resampled_S12 = resample(acc_S12, len(eda_S12))

# 🧩 Features kombinieren: EDA, TEMP, ACC → (n_samples, 6)
X_raw_S12 = np.hstack([eda_S12, temp_S12, acc_resampled_S12])

# 🪟 Sliding-Window Parameter
window_size_S12 = 384  # 60s Fenster
step_size_S12 = 384

features_S12 = []
window_labels_S12 = []

# 🚶 Sliding-Window-Verarbeitung
for start in range(0, len(X_raw_S12) - window_size_S12, step_size_S12):
    end = start + window_size_S12
    window_S12 = X_raw_S12[start:end]
    label_window_S12 = np.round(labels_resampled_S12[start:end]).astype(int)

    unique_labels_S12 = np.unique(label_window_S12)
    
    # ✅ Nur Fenster mit ausschließlich Label 1 oder 2
    if set(unique_labels_S12).issubset({1, 2}):
        majority_label_S12 = np.bincount(label_window_S12).argmax()
        feature_vector_S12 = np.concatenate([window_S12.mean(axis=0), window_S12.std(axis=0)])
        features_S12.append(feature_vector_S12)
        window_labels_S12.append(majority_label_S12)

# 🎯 Nur Label 1 & 2 verwenden
mask_S12 = (np.array(window_labels_S12) == 1) | (np.array(window_labels_S12) == 2)
X_S12 = np.array(features_S12)[mask_S12]
y_S12 = np.array(window_labels_S12)[mask_S12]

# ✅ Sicherheits-Check
assert len(X_S12) == len(y_S12), "Unterschiedliche Anzahl von Features und Labels!"

# ℹ️ Ergebnis anzeigen
print(f"Datensätze: {X_S12.shape}, Klassenverteilung: {np.bincount(y_S12)}")


Datensätze: (17, 10), Klassenverteilung: [ 0 11  6]


In [12]:
print(X_S12.shape, y_S12.shape)
print(X.shape, y.shape)

# ⬇️ Laden
rf_loaded = joblib.load("/kaggle/working/random_forest_model.pkl")

# Testen ob es klappt
print("TEST nach Laden", rf_loaded.predict(X_S12[:5]))

# Test auf Subject S12
y_pred = clf.predict(X_S12)
print(classification_report(y_S12, y_pred, digits=3))

(17, 10) (17,)
(227, 10) (227,)
TEST nach Laden [1 1 1 1 1]
              precision    recall  f1-score   support

           1      0.733     1.000     0.846        11
           2      1.000     0.333     0.500         6

    accuracy                          0.765        17
   macro avg      0.867     0.667     0.673        17
weighted avg      0.827     0.765     0.724        17



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def finde_bestes_n_estimators(X_train, X_test, y_train, y_test, 
                              n_min=10, n_max=200, step=10, average="weighted"):
    ergebnisse = {}
    
    for n in range(n_min, n_max+1, step):
        clf = RandomForestClassifier(n_estimators=n, random_state=42)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        acc = accuracy_score(y_test, y_pred)
        f1  = f1_score(y_test, y_pred, average=average)
        
        ergebnisse[n] = {"accuracy": acc, "f1_score": f1}
        #print(f"n_estimators={n}: Accuracy={acc:.3f}, F1={f1:.3f}")
    
    # Auswahl nach bestem F1-Score
    best_n = max(ergebnisse, key=lambda k: ergebnisse[k]["f1_score"])
    best_vals = ergebnisse[best_n]
    
    print(f"\n➡️ Bestes n_estimators: {best_n} "
          f"mit Accuracy={best_vals['accuracy']:.3f}, F1={best_vals['f1_score']:.3f}")
    
    return best_n, ergebnisse

print("Übersicht LOSO nur S8", X_S12.shape, y_S12.shape)
print("Übersicht WeSad der Rest(alle)",X.shape, y.shape)

# Setze X und Y (alle außer S8 ein) und schaue was der beste n_estimator ist
best_estimator = finde_bestes_n_estimators(X, X_S12, y, y_S12, n_min=10, n_max=200, step=1,average="weighted")

# Bestes n_estimators: 27 mit Accuracy=0.941
# Bestes n_estimators: 27 mit Accuracy=0.941, F1=0.940

Übersicht LOSO nur S8 (17, 10) (17,)
Übersicht WeSad der Rest(alle) (227, 10) (227,)

➡️ Bestes n_estimators: 27 mit Accuracy=0.941, F1=0.940


## Alternative zu Random Forest testen

## XGBoost

In [14]:
!pip install xgboost --quiet


In [15]:
import xgboost as xgb
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import classification_report
import numpy as np

# ✅ Filter: Nur Labels 1 (entspannt) & 2 (gestresst)
mask_train = (y == 1) | (y == 2)
mask_test = (y_S12 == 1) | (y_S12 == 2)

X_train_filtered = X[mask_train]
y_train_filtered = y[mask_train]

X_test_filtered = X_S12[mask_test]
y_test_filtered = y_S12[mask_test]


# Optional: Konvertiere in DMatrix (XGBoost-eigenes Format) – ist aber nicht zwingend
# dtrain = xgb.DMatrix(X, label=y)
# dtest = xgb.DMatrix(X_S12, label=y_S12)

# Mapping: 1 → 0 (Entspannt), 2 → 1 (Stress)
y_train_remap = (y_train_filtered - 1).astype(int)
y_test_remap = (y_test_filtered - 1).astype(int)


xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',  # da nur 2 Klassen nach dem Mapping
    n_estimators=200,
    max_depth=5,
    learning_rate=0.27,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='auc',
    random_state=42
)

xgb_clf.fit(X_train_filtered, y_train_remap)
y_pred = xgb_clf.predict(X_test_filtered)

# Evaluation wieder mit Original-Labels:
from sklearn.metrics import classification_report
print(classification_report(y_test_remap, y_pred, digits=3))


              precision    recall  f1-score   support

           0      0.909     0.909     0.909        11
           1      0.833     0.833     0.833         6

    accuracy                          0.882        17
   macro avg      0.871     0.871     0.871        17
weighted avg      0.882     0.882     0.882        17



-> Gutes Ergebnis:
Alle Personen außer S8; sonst alle drin:

              precision    recall  f1-score   support

           0      0.909     0.909     0.909        11
           1      0.833     0.833     0.833         6

    accuracy                          0.882        17
   macro avg      0.871     0.871     0.871        17
weighted avg      0.882     0.882     0.882        17

-> 5/6 Stressfälle erkannt. 
Lernrate manuell angepasst -> Automatisiert noch besser.

-> Interpretation: XGBoost besser als Random Forest, da Gradiant Boosting: Fehler aus vorherigen Bäumen finden + sich verbessern.

Es kann auch komplexere Zusammenhänge modellieren, wo Random Forest eher Mittelwerte bildet.

Boosting ist oft besser für kleine, saubere Zeitfenster-Daten als vorher.

Settings:
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',  # da nur 2 Klassen nach dem Mapping
    n_estimators=200,
    max_depth=5,
    learning_rate=0.27,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)





## Modell speichern

In [16]:
import joblib

# Modell speichern
joblib.dump(xgb_clf, "/kaggle/working/xgb_stress_model.pkl")

['/kaggle/working/xgb_stress_model.pkl']

## Herausfinden, welche Parameter am besten geeignet sind

In [17]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

def finde_beste_xgb_parameter(X_train, X_test, y_train, y_test, param_grid, average="weighted"):
    ergebnisse = []
    
    for params in ParameterGrid(param_grid):
        clf = xgb.XGBClassifier(
            objective='binary:logistic',
            use_label_encoder=False,
            eval_metric='auc',
            random_state=42,
            **params
        )
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        acc = accuracy_score(y_test, y_pred)
        f1  = f1_score(y_test, y_pred, average=average)
        
        ergebnisse.append((params, acc, f1))
        #print(f"Params={params} | Accuracy={acc:.3f}, F1={f1:.3f}")
    
    # Sortierung nach F1, dann Accuracy
    best_params, best_acc, best_f1 = max(ergebnisse, key=lambda x: (x[2], x[1]))
    
    print(f"\n➡️ Beste Parameter: {best_params}")
    print(f"   Accuracy={best_acc:.3f}, F1={best_f1:.3f}")
    
    return best_params, best_acc, best_f1


In [18]:
X_train_filtered = X[mask_train]
y_train_filtered = y[mask_train]

X_train_filtered = (X_train_filtered - 1).astype(int)
y_train_filtered = (y_train_filtered - 1).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X_train_filtered, y_train_filtered, 
    test_size=0.2,     # 20% Testdaten
    random_state=42,   # Reproduzierbarkeit
)
param_grid = {
    "n_estimators": [10, 20, 30, 40, 50, 60,70, 80, 100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.05, 0.1, 0.2],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

beste_params, beste_acc, beste_f1 = finde_beste_xgb_parameter(
    X_train, X_test, y_train, y_test, param_grid
)



➡️ Beste Parameter: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}
   Accuracy=0.935, F1=0.933


## Testen mit S8 als LOSO Verfahren

In [19]:
X_test = X_S12[mask_test]
y_test = y_S12[mask_test]

X_train, X_test, y_train, y_test = train_test_split(
    X_train_filtered, y_train_filtered, 
    test_size=0.2,     # 20% Testdaten
    random_state=42,   # Reproduzierbarkeit
)
param_grid = {
    "n_estimators": [10, 20, 30, 40, 50, 60,70, 80, 100, 200, 300],
    "max_depth": [3, 5, 7, 9, 11, 13, 15],
    "learning_rate": [0.05, 0.1,0.15,0.2, 0.25, 0.30],
    "subsample": [0.6,0.7,0.8,0.9, 1.0],
    "colsample_bytree": [0.6, 0.7,0.8,0.9,1.0]
}

beste_params, beste_acc, beste_f1 = finde_beste_xgb_parameter(
    X_train, X_test, y_train, y_test, param_grid
)



➡️ Beste Parameter: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}
   Accuracy=0.935, F1=0.933


## SVM ("Support Vector Maschines")


In [20]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np

# 🔄 Nur Labels 1 und 2 (wie bei XGBoost)
mask_train = (y == 1) | (y == 2)
mask_test = (y_S12 == 1) | (y_S12 == 2)

X_train_filtered = X[mask_train]
y_train_filtered = y[mask_train]
X_test_filtered = X_S12[mask_test]
y_test_filtered = y_S12[mask_test]

# 🔄 Labels ummappen: 1 → 0, 2 → 1
y_train_remap = (y_train_filtered - 1).astype(int)
y_test_remap = (y_test_filtered - 1).astype(int)

# 🔄 Features skalieren (wichtig für SVM!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered)
X_test_scaled = scaler.transform(X_test_filtered)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1, 'scale', 'auto'],
    'kernel': ['rbf']
}

grid = GridSearchCV(SVC(), param_grid, cv=3, scoring='f1')
grid.fit(X_train_scaled, y_train_remap)

print("Beste Parameter:", grid.best_params_)
print("")

# ✅ SVM mit RBF-Kernel
svm_clf = SVC(kernel='rbf', C=10, gamma=0.1, random_state=42)
svm_clf.fit(X_train_scaled, y_train_remap)

# 🧪 Vorhersage & Bewertung
y_pred = svm_clf.predict(X_test_scaled)
print(classification_report(y_test_remap, y_pred, digits=3))


Beste Parameter: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

              precision    recall  f1-score   support

           0      0.400     0.182     0.250        11
           1      0.250     0.500     0.333         6

    accuracy                          0.294        17
   macro avg      0.325     0.341     0.292        17
weighted avg      0.347     0.294     0.279        17



In [21]:
# Modell speichern
import joblib

# Modell speichern
joblib.dump(svm_clf, "/kaggle/working/svm_clf_stress_model.pkl")

['/kaggle/working/svm_clf_stress_model.pkl']

## CNN 

In [22]:
import pandas as pd
import numpy as np
from scipy.signal import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

file_paths = [
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S14/S14.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S13/S13.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S10/S10.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S5/S5.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S7/S7.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S9/S9.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S15/S15.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S2/S2.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S6/S6.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S3/S3.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S4/S4.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S16/S16.pkl',
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S17/S17.pkl'
]

X_all_list = []
y_all_list = []

for file in file_paths:
    with open(file, 'rb') as f:
        data = pd.read_pickle(f)

    wrist = data['signal']['wrist']
    eda = wrist['EDA']
    temp = wrist['TEMP']
    acc = wrist['ACC']
    labels = data['label']

    # Resample Labels auf EDA-Länge
    labels_resampled = resample(labels.astype(float), len(eda))
    labels_resampled = np.round(labels_resampled).astype(int)

    # ACC resamplen und alle Kanäle kombinieren
    acc_resampled = resample(acc, len(eda))
    X_raw = np.hstack([eda, temp, acc_resampled])  # shape: (n_samples, 5)

    # Sliding Window
    window_size = 384
    step_size = 384

    for start in range(0, len(X_raw) - window_size, step_size):
        end = start + window_size
        window = X_raw[start:end]
        label_window = labels_resampled[start:end]

        if set(np.unique(label_window)).issubset({1, 2}):
            X_all_list.append(window)
            majority_label = np.bincount(label_window).argmax()
            y_all_list.append(majority_label)

# In Arrays umwandeln
X_all = np.array(X_all_list)                   # shape: (n_windows_total, 384, 5)
y_all = (np.array(y_all_list) - 1).astype(int) # [1,2] → [0,1]

print(f"Datensätze: {X_all.shape}, Klassenverteilung: {np.bincount(y_all)}")


Datensätze: (227, 384, 5), Klassenverteilung: [149  78]


In [23]:
y_cnn = (y_all - 1).astype(int)  # [1, 2] → [0, 1]
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.2, stratify=y_cnn, random_state=42
)
X_train

array([[[ 9.22620000e-01,  3.35900000e+01,  5.57359754e+01,
         -1.16405106e+01, -2.36790809e+01],
        [ 9.02153000e-01,  3.35900000e+01,  5.90941376e+01,
         -1.22361185e+01, -2.86766943e+01],
        [ 8.98316000e-01,  3.35700000e+01,  5.96143522e+01,
         -1.19007638e+01,  1.51951757e+00],
        ...,
        [ 1.00192900e+00,  3.33400000e+01,  6.20531689e+01,
         -1.09214411e+01, -4.47238530e+00],
        [ 1.00320800e+00,  3.33400000e+01,  6.21427682e+01,
         -1.10735819e+01, -4.67169647e+00],
        [ 1.00832500e+00,  3.33400000e+01,  6.18994900e+01,
         -1.09335418e+01, -5.03437713e+00]],

       [[ 8.87166600e+00,  3.47900000e+01,  2.00651808e+01,
          5.46417869e+00,  6.02469766e+01],
        [ 8.83968400e+00,  3.47900000e+01,  1.97015015e+01,
          4.49243741e+00,  5.91979346e+01],
        [ 8.81793800e+00,  3.47900000e+01,  2.02628546e+01,
          5.03709956e+00,  5.94010583e+01],
        ...,
        [ 8.11995000e+00,  3.4710000

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv1D(32, kernel_size=5, activation='relu', input_shape=(384, 5)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


2025-09-19 11:14:28.711479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758280468.937938      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758280469.004989      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-19 11:14:43.238656: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 380, 32)        │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 190, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 188, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 94, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6016)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       385,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,193 (1.50 MB)

 Trainable params: 392,193 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - accuracy: 0.6359 - loss: 5.6820 - val_accuracy: 0.7297 - val_loss: 1.8296
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7843 - loss: 4.1923 - val_accuracy: 0.8649 - val_loss: 1.4169
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7386 - loss: 2.4382 - val_accuracy: 0.7297 - val_loss: 2.0824
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8082 - loss: 1.0423 - val_accuracy: 0.7027 - val_loss: 1.3431
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8464 - loss: 0.3829 - val_accuracy: 0.8108 - val_loss: 0.5932
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8475 - loss: 0.2620 - val_accuracy: 0.8378 - val_loss: 0.4231
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8976 - loss: 0.2526 - val_accuracy: 0.8919 - val_loss: 0.3946
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9029 - loss: 0.2250 - val_accuracy: 0.8649 - val_loss: 0.3987

In [26]:
model.save("/kaggle/working/cnn_model.keras")


In [27]:
from tensorflow.keras.models import load_model

# Laden (egal ob .keras oder .h5)
cnn_loaded = load_model("/kaggle/working/cnn_model.keras")

# Test
print(cnn_loaded.predict(X_test[:5]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
[[1.8849039e-03]
 [7.2859526e-01]
 [1.8299589e-01]
 [1.2828547e-16]
 [5.0316366e-23]]


In [28]:
## Download der Daten
from IPython.display import FileLink

FileLink('/kaggle/working/cnn_model.keras')
FileLink('/kaggle/working/svm_clf_stress_model.pkl')


/kaggle/working/svm_clf_stress_model.pkl

In [29]:
import os, zipfile, shutil

# 1) Sicherstellen, dass die Modelle da sind
files_to_zip = [
    "/kaggle/working/cnn_model.keras",
    "/kaggle/working/random_forest_model.pkl",
    "/kaggle/working/svm_clf_stress_model.pkl",
]

missing = [p for p in files_to_zip if not os.path.exists(p)]
if missing:
    print("Fehlende Dateien:", missing)
else:
    print("Alle Dateien vorhanden.")

# 2) Sauberen Export-Ordner anlegen
export_dir = "/kaggle/working/export"
os.makedirs(export_dir, exist_ok=True)

# 3) Dateien in den Export-Ordner kopieren (optional, hält’s übersichtlich)
for p in files_to_zip:
    if os.path.exists(p):
        shutil.copy(p, os.path.join(export_dir, os.path.basename(p)))

# 4) Zip nur mit diesen Dateien bauen
zip_path = "/kaggle/working/models.zip"
with zipfile.ZipFile(zip_path, "w") as zf:
    for name in os.listdir(export_dir):
        zf.write(os.path.join(export_dir, name), arcname=name)

# 5) Kontrolle: auflisten und Größen anzeigen
for name in os.listdir("/kaggle/working"):
    full = os.path.join("/kaggle/working", name)
    if os.path.isfile(full):
        print(f"{name}: {os.path.getsize(full)/1024:.1f} KB")

print("\nFERTIG: models.zip liegt in /kaggle/working und erscheint im Output-Tab.")


Alle Dateien vorhanden.
xgb_stress_model.pkl: 167.1 KB
__notebook__.ipynb: 157.4 KB
random_forest_model.pkl: 316.2 KB
svm_clf_stress_model.pkl: 9.8 KB
cnn_model.keras: 4634.6 KB
models.zip: 4960.9 KB

FERTIG: models.zip liegt in /kaggle/working und erscheint im Output-Tab.


## Ausprobieren CNN für "gelernte Daten" 

In [30]:
from sklearn.metrics import classification_report

y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred, digits=3))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step
              precision    recall  f1-score   support

           0      0.962     0.833     0.893        30
           1      0.750     0.938     0.833        16

    accuracy                          0.870        46
   macro avg      0.856     0.885     0.863        46
weighted avg      0.888     0.870     0.872        46



## Ausprobieren CNN für "NEUE Daten" -> 8

-> Variablen ändern für S12 bzw. S8 
-> Ausführen
-> 

In [31]:
import pandas as pd
import numpy as np
from scipy.signal import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

file_paths = [
    '/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S8/S8.pkl'
]

X_all_list_S8 = []
y_all_list_S8 = []

for file in file_paths:
    with open(file, 'rb') as f:
        data = pd.read_pickle(f)

    wrist = data['signal']['wrist']
    eda = wrist['EDA']
    temp = wrist['TEMP']
    acc = wrist['ACC']
    labels = data['label']

    # Resample Labels auf EDA-Länge
    labels_resampled = resample(labels.astype(float), len(eda))
    labels_resampled = np.round(labels_resampled).astype(int)

    # ACC resamplen und alle Kanäle kombinieren
    acc_resampled = resample(acc, len(eda))
    X_raw = np.hstack([eda, temp, acc_resampled])  # shape: (n_samples, 5)

    # Sliding Window
    window_size = 384
    step_size = 384

    for start in range(0, len(X_raw) - window_size, step_size):
        end = start + window_size
        window = X_raw[start:end]
        label_window = labels_resampled[start:end]

        if set(np.unique(label_window)).issubset({1, 2}):
            X_all_list_S8.append(window)
            majority_label_S8 = np.bincount(label_window).argmax()
            y_all_list_S8.append(majority_label_S8)

# In Arrays umwandeln
X_S8 = np.array(X_all_list_S8)                   # shape: (n_windows_total, 384, 5)
y_S8 = (np.array(y_all_list_S8) - 1).astype(int) # [1,2] → [0,1]

print(f"Datensätze: {X_S8.shape}, Klassenverteilung: {np.bincount(y_S8)}")


Datensätze: (17, 384, 5), Klassenverteilung: [11  6]


In [32]:
y_cnn = (y_S8 - 1).astype(int)  # [1, 2] → [0, 1]

#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


from sklearn.metrics import classification_report

y_pred_prob = model.predict(X_S8).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_S8, y_pred, digits=3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
              precision    recall  f1-score   support

           0      0.333     0.273     0.300        11
           1      0.000     0.000     0.000         6

    accuracy                          0.176        17
   macro avg      0.167     0.136     0.150        17
weighted avg      0.216     0.176     0.194        17



In [33]:
file='/kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/S8/S8.pkl'

with open(file, 'rb') as f:
    data = pd.read_pickle(f)
print("Top-Level Keys:", data.keys())


Top-Level Keys: dict_keys(['signal', 'label', 'subject'])


In [34]:
print(data['signal']['chest'].keys())



dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])
